In [11]:
import utils.yolo_fun as yolo_fun
import utils.img_fun as img_fun
import os
import pandas as pd
from tqdm import tqdm  
import rasterio
from rasterio.windows import Window
from rasterio.errors import RasterioIOError
import numpy as np
import shutil

#valid_tiles = [13, 14, 15, 22, 23, 24, 35, 44, 45, 51, 52, 53, 54, 61, 62, 63, 70, 71, 72, 73, 81, 82]  

orthomosiac_coords = os.path.join('coords', 'yolo_coords.csv')
coords_dir_negatives = os.path.join('coords', 'negatives')

os.makedirs(coords_dir_negatives, exist_ok=True)
os.makedirs(coords_dir_normalized, exist_ok=True)

In [7]:

# Tile 24 tiene focas y pinguinos descansando

for n_tile in range(24, 25):
    try:
        # PARTE 1: Cargar la imagen y recortarla en imágenes más pequeñas de aproximadamente 500x500 píxeles
        print(f"\n\nRecortando imagen {n_tile}...")
        print('_________________________________________________________')
        NUM_TILE = n_tile
        path_doctorado = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos'
        subrecortes_dir = os.path.join('cut_tiles', 'negatives')
        os.makedirs(subrecortes_dir, exist_ok=True)

        image_name = f"recortes/recorte_{NUM_TILE}.tif"
        tiff_file = os.path.join(path_doctorado, image_name)


        # Sacamos un diccionario con toda la información de la imagen
        img_info = img_fun.get_img_info(tiff_file)
        WIDTH = img_info["width"]
        HEIGHT = img_info["height"]
        TOP_LEFT = img_info["top_left"]
        BOTTOM_RIGHT = img_info["bottom_right"]
        min_x, max_y = img_info['top_left']
        max_x, min_y = img_info['bottom_right']

        img_fun.crop_tile_into_subrecortes(
            tiff_file = tiff_file, 
            output_dir = subrecortes_dir, 
            num_tile = NUM_TILE,
            is_negative = True
        )




    except RasterioIOError as e:
        print(f"Error al cargar la imagen con rasterio: {e}")
        continue
    except FileNotFoundError as e:
        print(f"Error: {e}")
        continue
    except Exception as e:
        print(f"Error: {e}")
        continue



Recortando imagen 24...
_________________________________________________________
Metadata:
---------
driver: GTiff
dtype: uint8
nodata: None
width: 10195
height: 11420
count: 4
crs: EPSG:4326
transform: | 0.00, 0.00,-59.23|
| 0.00,-0.00,-62.29|
| 0.00, 0.00, 1.00|
blockxsize: 10195
blockysize: 1
tiled: False
interleave: pixel

Coordenadas de las esquinas de la imagen:
TOP LEFT: (-59.22587875249678, -62.290013442834734)
BOTTOM RIGHT: (-59.22062349506678, -62.29275364899473)
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coordenadas vacías... guardando
Coorden

### Creación de las etiquetas txt

Después de seleccionar exactamente los negativos que queremos, ahora sí que podemos generar los txt. Puesto que los txt están vacíos, bastará con pegarlos en nuestra carpeta de datasets y ya estarán listos para ser usados.

Para tenerlos mejor identificados, los 20 primeros (o 30 o 40, si tenemos más) -con cierta variedad- serán para el conjunto de validación. El resto, para train.

In [12]:
 # PARTE 3: ASIGNACIÓN DE LABELS EN TXT A CADA SUBRECORTE
for img in os.listdir(subrecortes_dir):
    if not img.lower().endswith(('.tif', '.tiff', '.jpg', '.png')):
        continue

    txt_name = img.split('.')[0] + '.txt'
    txt_path = os.path.join(coords_dir_negatives, txt_name)
    with open(txt_path, 'w') as f:
        f.write("")  